In [1]:
import sys
import random
import numpy as np
from scipy.misc import imread
import tensorflow as tf
 
num = 2000
    
img_list = ['zero_'+str(i)+'.png' for i in range(1,num+1)]
lbl_list = [0 for i in range(num)]
img_list = img_list+['one_'+str(i)+'.png' for i in range(1,num+1)]
lbl_list = lbl_list+[1 for i in range(num)]

# print img_list,lbl_list

In [2]:
def data_provider(img_list,lbl_list,n):
    
    images = np.zeros((n,28,28,1))
    labels = np.zeros((n,2))
    
    n_files = len(img_list)
    n = min(n,n_files)
    
    lst = np.arange(n_files)
    random.shuffle(lst)
    lst = lst[:n]
    
    for i in range(n):
        images[i,:,:,0] = imread('./media/MNIST_images/'+img_list[lst[i]])
        labels[i,lbl_list[lst[i]]] =1 
        
    return images,labels

In [31]:
x_in = tf.placeholder(tf.float32, shape=[None,28,28,1])
y_ = tf.placeholder(tf.float32, shape=[None,2])
keep_prob = tf.placeholder(tf.float32)

x = tf.layers.conv2d(x_in,filters=5,kernel_size=3,strides=(1, 1),padding='same',
            activation=tf.nn.relu)
x = tf.layers.max_pooling2d(x,pool_size=2,strides=2)

x = tf.layers.conv2d(x,filters=5,kernel_size=3,strides=(1, 1),padding='same',
        activation=tf.nn.relu)
x = tf.layers.max_pooling2d(x,pool_size=2,strides=2)

x = tf.contrib.layers.flatten(x)
x = tf.nn.dropout(x, keep_prob)

y_conv = tf.layers.dense(x,2,activation=tf.nn.softmax)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [35]:
def train(img_list,lbl_list,restore=1,model_add='model'):
    sess = tf.InteractiveSession()
    saver = tf.train.Saver()

    if restore:
#         tf.reset_default_graph()
        saver.restore(sess, model_add+'/model')
    else:
        init = tf.global_variables_initializer()
        sess.run(init)

    for i in range(100):
        xp,yp = data_provider(img_list,lbl_list,100)
        if i % 10 == 0:
            train_accuracy = accuracy.eval(feed_dict={x_in: xp, y_: yp, keep_prob: 1.0})
            print('{:d}, {:2.2f}'.format(i, train_accuracy))
        train_step.run(feed_dict={x_in: xp, y_: yp, keep_prob: 0.8})

    saver.save(sess, model_add+'/model')
    
train(img_list,lbl_list,restore=1)

INFO:tensorflow:Restoring parameters from model/model
0, 0.92
10, 0.97
20, 0.94
30, 0.95
40, 0.95
50, 0.99
60, 0.99
70, 0.99
80, 0.95
90, 0.97


In [43]:
def predict(img_path,model_add='model'):
    images = np.zeros((1,28,28,1))
    images[0,:,:,0] = imread(img_path)

    sess = tf.InteractiveSession()
    saver = tf.train.Saver()
    saver.restore(sess, model_add+'/model')

    y_out = sess.run(y_conv,feed_dict={x_in: images, keep_prob: 1.0})
    return np.argmax(y_out)

In [44]:
img_path = './media/MNIST_images/zero_15.png'
predict(img_path,model_add='model')

INFO:tensorflow:Restoring parameters from model/model


0